## 引入所需套件

In [88]:
import urllib.request as req
import bs4 as bs
import urllib.request as req
import urllib.parse
import pandas as pd

## 開始爬蟲

In [68]:
def get_html_from_url(url):
    r = req.Request(url)
    r.add_header("User-Agent", "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36")
    resp = req.urlopen(r)
    html = bs.BeautifulSoup(resp)
    return html

In [86]:
def crawl_a_job(job_link):
    html = get_html_from_url(job_link)

    description = html.find('div', {'class': 'job_explain mt'})
    ul_item = description.find('ul')
    li_items = ul_item.find_all('li')

    # 我們需要的項目（還沒爬可怕的工作內容、寫在法定保障（而非休假制度項目）的休假制度）
    map_dict = {
        '上班日期': 'work_time',
        '工作性質': 'work_type',
        '工作地點': 'address'  # 我把location視為區域，address視為地址，因此實質意義是公司地址
    }

    job_dict = {}

    # 還是保留個空值
    for key, value in map_dict.items():
        job_dict[value] = ''

    for li_item in li_items:
        # 先爬取這個項目的副標題
        left_title = li_item.find('span', {'class': 'left_title'})
        if left_title is None:
            continue
        left_title_text = left_title.get_text()
        # 再一一核對職缺說明有沒有這一項
        for key, value in map_dict.items():
            if key in left_title_text:
                right_main_text = li_item.find('span', {'class': 'right_main'}).get_text()
                job_dict[value] = right_main_text.strip()

    return job_dict

In [87]:
def crawl_all_results(html):
    job_items = html.find_all("div",{"class":"Job_opening_M"})
    print(len(job_items))

    job_list = []

    for job_item in job_items:
        print('---------------------------')
        divs = job_item.find("div",{"class":"Job_opening_item_info_M"}).find_all('div')
        # 工作名稱
        title_text = divs[1].find('h5').get_text().strip()
        # 以下兩個之後會在table「company」獨立建表
        company_text = divs[2].find('h5').get_text().strip()
        # 地區
        location_text = divs[3].find('h5').get_text().strip()
        salary_text = divs[4].find('h5').get_text().strip()

        # tags: 有些有有些沒有
        tags = ''
        if len(divs) > 5:
            for div in divs[5:]:
                tags += div.get_text().strip()

        job_dict = {
            "title": title_text,
            "salary": salary_text,
            "company": company_text,
            "location": location_text,
            "tags": tags
        };
        print(job_dict)

        # 準備爬取每個職缺連結
        job_link = job_item.find('a')['href']
        job_link = 'https://www.yes123.com.tw/wk_index/' + job_link
        print(job_link)

        # 爬取頁面所需資訊（未完）
        page_dict = crawl_a_job(job_link)
        print(page_dict)

        # 結合主頁面和單一工作頁面所得資訊，加入串列中
        job_dict = job_dict | page_dict
        job_list.append(job_dict)

    return job_list

In [71]:
find_work_mode1 = "2_1011_0001_0017"
s_find_work_mode1 = "資料工程師"

# 原始 URL，含有中文
url = f'https://www.yes123.com.tw/wk_index/joblist.asp?find_work_mode1={find_work_mode1}&s_find_work_mode1={s_find_work_mode1}&order_ascend=desc&strrec=0&search_type=job&search_from=joblist'

# 將中文部分進行編碼
parsed = urllib.parse.urlsplit(url)
encoded_path = urllib.parse.quote(parsed.path)
encoded_query = urllib.parse.quote(parsed.query, safe="=&")
# 組合回完整 URL
url = urllib.parse.urlunsplit((parsed.scheme, parsed.netloc, encoded_path, encoded_query, parsed.fragment))
print(url)

https://www.yes123.com.tw/wk_index/joblist.asp?find_work_mode1=2_1011_0001_0017&s_find_work_mode1=%E8%B3%87%E6%96%99%E5%B7%A5%E7%A8%8B%E5%B8%AB&order_ascend=desc&strrec=0&search_type=job&search_from=joblist


In [81]:
if __name__ == '__main__':
    html = get_html_from_url(url)
    job_dict = crawl_all_results(html)
    # print(job_dict)

26
{'title': 'AI工程師', 'salary': '月薪\u300040,000 至 45,000元', 'company': '快樂島股份有限公司', 'location': '台北市內湖區', 'tags': '#免經驗\n#二度就業\n#彈性上下班'}
https://www.yes123.com.tw/wk_index/job.asp?p_id=20250216110109_53743163&job_id=20250315224637_9947117
{'work_type': '全職', 'address': '台北市內湖區瑞光路335號3樓(內湖科技園區)'}


In [89]:
html = get_html_from_url(url)
job_list = crawl_all_results(html)

26
---------------------------
{'title': 'AI工程師', 'salary': '月薪\u300040,000 至 45,000元', 'company': '快樂島股份有限公司', 'location': '台北市內湖區', 'tags': '#免經驗\n#二度就業\n#彈性上下班'}
https://www.yes123.com.tw/wk_index/job.asp?p_id=20250216110109_53743163&job_id=20250315224637_9947117
{'work_time': '', 'work_type': '全職', 'address': '台北市內湖區瑞光路335號3樓(內湖科技園區)'}
---------------------------
{'title': 'D4000Y 資料倉儲數據工程師', 'salary': '薪資面議(經常性薪資達4萬元含以上)', 'company': '富邦媒體科技股份有限公司', 'location': '台北市內湖區', 'tags': ''}
https://www.yes123.com.tw/wk_index/job.asp?p_id=20140103100924_27365925&job_id=20240328032557_23778114
{'work_time': '一個月內', 'work_type': '全職', 'address': '台北市內湖區(依公司規定)'}
---------------------------
{'title': '資訊課級主管(資料分析與架構研發)', 'salary': '薪資面議(經常性薪資達4萬元含以上)', 'company': '富邦媒體科技股份有限公司', 'location': '台北市內湖區', 'tags': ''}
https://www.yes123.com.tw/wk_index/job.asp?p_id=20140103100924_27365925&job_id=20250313033103_1840801
{'work_time': '隨 時', 'work_type': '全職', 'address': '台北市內湖區洲子街96號4樓(內湖科技園區)'}
----

## 轉換成dataframe

In [91]:
df = pd.DataFrame(job_list)
df

,title,salary,company,location,tags,work_time,work_type,address
0,AI工程師,"月薪 40,000 至 45,000元",快樂島股份有限公司,台北市內湖區,#免經驗\n#二度就業\n#彈性上下班,,全職,台北市內湖區瑞光路335號3樓(內湖科技園區)
1,D4000Y 資料倉儲數據工程師,薪資面議(經常性薪資達4萬元含以上),富邦媒體科技股份有限公司,台北市內湖區,,一個月內,全職,台北市內湖區(依公司規定)
2,資訊課級主管(資料分析與架構研發),薪資面議(經常性薪資達4萬元含以上),富邦媒體科技股份有限公司,台北市內湖區,,隨 時,全職,台北市內湖區洲子街96號4樓(內湖科技園區)
3,☆永科☆日商科技廠＜正職＞資訊人員,"月薪 35,000 至 43,000元",萬通國際人力開發股份有限公司,台南市永康區,#有獎金分紅\n#年終獎金\n#24h必回覆,隨 時,全職,台南市永康區永科環路
4,D4000 資訊課級主管(客戶行為應用方向),薪資面議(經常性薪資達4萬元含以上),富邦媒體科技股份有限公司,台北市內湖區,,隨 時,全職,台北市內湖區洲子街96號4樓(內湖科技園區)
5,廣告平台資料工程師,薪資面議(經常性薪資達4萬元含以上),富邦媒體科技股份有限公司,台北市內湖區,,隨 時,全職,台北市內湖區(依公司規定)
6,D4000 資料科學家,薪資面議(經常性薪資達4萬元含以上),富邦媒體科技股份有限公司,台北市內湖區,,一個月內,全職,台北市內湖區洲子街內湖科技園區(內湖科技園區)
7,廣告推薦機器學習工程師,薪資面議(經常性薪資達4萬元含以上),富邦媒體科技股份有限公司,台北市內湖區,,隨 時,全職,台北市內湖區(依公司規定)
8,【總公司_資訊部】管理師,薪資面議(經常性薪資達4萬元含以上),新東陽股份有限公司,台北市大安區,,兩週內,全職,台北市大安區忠孝東路四段289號8樓
9,資料架構工程師,薪資面議(經常性薪資達4萬元含以上),富邦媒體科技股份有限公司,台北市內湖區,,隨 時,全職,台北市內湖區洲子街96號4樓(內湖科技園區)


## 備用：如果需要使用 selenium

### 安裝所需套件

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

### 載入套件

In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.headless = True
driver = webdriver.Chrome(options=chrome_options)

### 課堂範例用法

In [ ]:
driver.get("https://www.google.com")
driver.maximize_window()
# 開始使用selenium
# 常見語法：BY.TAG_NAME、BY.CLASS_NAME、BY.ID
# 尋找Chrome首頁的文字輸入框
e = driver.find_element(By.CLASS_NAME, "gLFyf")
print(e.get_attribute("class"))
# 點擊：click()，鍵盤輸入：send_keys("aaa")
e.send_keys("chiikawa")
e.send_keys(Keys.ENTER)

gLFyf
